## Site Access Score Tool

### May 26th, 2022
<br>

--------------------------------------------------------------------------------------------


#### Round 1

* GetParameterAsText (https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/getparameterastext.htm)
* SpatialReference (https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/spatialreference.htm)
* PointGeometry (https://pro.arcgis.com/en/pro-app/2.8/arcpy/classes/pointgeometry.htm)
* Copy Features (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/copy-features.htm)
* Buffer Analysis (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/buffer.htm)
* Intersect Analysis (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/intersect.htm)
* os.path.join() (https://www.geeksforgeeks.org/python-os-path-join-method/)

1) Created a point with longitude, latitude number (double) <br>
2) Created a buffer with a customized distance (linear unit) <br> 
3) Created point feature layers intersected between the buffer and major roads <br> 


--------------------------------------------------------------------------------------------


#### Round 2

* OriginDestinationCostMatrix (https://pro.arcgis.com/en/pro-app/2.8/arcpy/network-analyst/odcostmatrix.htm)
* Join Field (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/join-field.htm)
* Copy (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/copy.htm)
* Table to Excel (Conversion) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/conversion/table-to-excel.htm)
* Summary Statistics (Analysis) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/summary-statistics.htm)
* UpdateCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/updatecursor-class.htm)
  * SearchCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/searchcursor-class.htm)
  * InsertCursor (https://pro.arcgis.com/en/pro-app/2.8/arcpy/data-access/insertcursor-class.htm)
* AddMessage (https://pro.arcgis.com/en/pro-app/2.8/arcpy/functions/addmessage.htm)
* Delete (Data Management) (https://pro.arcgis.com/en/pro-app/2.8/tool-reference/data-management/delete.htm)

1) Added workspace as a parameter for different environment settings <br>
2) Created drive times(minutes) and network distance(miles) from buffered points intersected with major roads to a target point <br>
3) Used our Network Dataset, not esri's one <br>
4) Exported excel(xlsx) files for the OD Cost Matrix as well as the summary calculation. <br>
5) Added each point's drive time and network distance to the result message. <br>


--------------------------------------------------------------------------------------------


#### Next Step - Round 3 (by June 13)

1) Multiple Input Locations & Buffer Distance with csv file
>| Tips |
<br>
* Use csv for the location inputs (e.g. Fields = ["ID", "Location", "Lat", "Long", "Buffer_Distance"]) <br>
* Need to change input parameter as csv file and use "for loop" to use all rows as inputs <br>
>

2) Calculate the number of turns
>| Tips |
<br>
* Copy Traversed Source Features (https://desktop.arcgis.com/en/arcmap/latest/tools/network-analyst-toolbox/copy-traversed-source-features.htm)
>


In [ ]:
# import system modules
import arcpy
from arcpy import env
import os
arcpy.CheckOutExtension("network")

# set environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = True

# set parameters
work_dbs = arcpy.GetParameterAsText(0) # workspace
in_lat = arcpy.GetParameterAsText(1) # Double, Y
in_long = arcpy.GetParameterAsText(2) # Double, X
point_target = arcpy.GetParameterAsText(3) # feature layer - Target Point
point_buffer = arcpy.GetParameterAsText(4) # feature layer - Buffer Area
buffer_dist = arcpy.GetParameterAsText(5) # Linear unit - Buffer Distance
output_point = arcpy.GetParameterAsText(6) # feature layer
output_point_xlsx = arcpy.GetParameterAsText(7) # xlsx
output_table_xlsx = arcpy.GetParameterAsText(8) # xlsx

# define workspace
arcpy.env.workspace = work_dbs
workspace = work_dbs

# create a point with spatial reference
point = arcpy.Point(in_long,in_lat)
sr = arcpy.SpatialReference("WGS 1984")
point_geom = arcpy.PointGeometry(point,sr)
# create a point feature class
arcpy.CopyFeatures_management(point_geom, point_target)

# create a buffer with the point and buffer distance
arcpy.Buffer_analysis(point_target, point_buffer, buffer_dist)

# create intersections between the buffer and major roads feature layer
  # Set local variable
major_roads = "MajorRoads"
intsct_point = os.path.join(workspace, "intsct_point")
arcpy.Intersect_analysis([point_buffer, major_roads], intsct_point, "", "", "point")

# set local variables to perform origin destination cost matrix
nds = "C:/ArcGIS/Business Analyst/US_2021/Data/Streets Data/NorthAmerica.gdb/Routing/Routing_ND"
nd_layer_name = "Routing_ND"
input_origin = intsct_point
input_destination = point_target

# Create a network dataset layer and get the desired travel mode for analysis
arcpy.nax.MakeNetworkDatasetLayer(nds, nd_layer_name)
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Driving Time"]

# Instantiate a OriginDestinationCostMatrix solver object
odcm = arcpy.nax.OriginDestinationCostMatrix(nd_layer_name)
# Set properties
odcm.travelMode = travel_mode
odcm.timeUnits = arcpy.nax.TimeUnits.Minutes
odcm.defaultDestinationCount = 1
odcm.distanceUnits = arcpy.nax.DistanceUnits.Miles
odcm.lineShapeType = arcpy.nax.LineShapeType.StraightLine
# Load inputs
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Origins, input_origin)
odcm.load(arcpy.nax.OriginDestinationCostMatrixInputDataType.Destinations, input_destination)
# Solve the analysis
result = odcm.solve()

# Export the results to a feature class
output_lines = os.path.join(workspace, "output_lines")
result.export(arcpy.nax.OriginDestinationCostMatrixOutputDataType.Lines, output_lines)

# Table join with the point feuatres 
field = "OBJECTID"
arcpy.JoinField_management(intsct_point, field, 
                           output_lines, field)
arcpy.Copy_management(intsct_point, output_point)

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_point, output_point_xlsx)

# Execute Summary Statistics and create a table
output_table = os.path.join(workspace, "output_table")
arcpy.Statistics_analysis(
    in_table = output_lines, 
    out_table = output_table, 
    statistics_fields = [["Total_Time", "MEAN"],
                         ["Total_Time", "MAX"],
                         ["Total_Time", "MIN"],
                         ["Total_Time", "MEDIAN"],
                         ["Total_Time", "RANGE"],
                         ["Total_Distance", "MEAN"],
                         ["Total_Distance", "MAX"],
                         ["Total_Distance", "MIN"],
                         ["Total_Distance", "MEDIAN"],
                         ["Total_Distance", "RANGE"],
                        ])

# Export from feature class table to excel
arcpy.TableToExcel_conversion(output_table, output_table_xlsx)

# Use ORDER BY sql clause to sort field values
fields = ['OID@', 'Total_Time', 'Total_Distance']
with arcpy.da.UpdateCursor(
    in_table = output_lines, 
    field_names = fields, 
    sql_clause=(None, 'ORDER BY Total_Time DESC')) as cursor:
    for row in cursor:
        row[1] = round(float(row[1]), 1)
        row[2] = round(float(row[2]), 2)
        cursor.updateRow(row)
        arcpy.AddMessage('ID: {}, Drive Time: {} Minutes, Drive Distance: {} Miles'.format(row[0], row[1], row[2]))
    
# Delete unnecessary data
arcpy.management.Delete(intsct_point)
arcpy.management.Delete(output_lines)
arcpy.management.Delete(output_table)
